In [1]:
import ee
ee.Initialize()

In [2]:
template =  '''<GDAL_WMS>
    <Service name="TMS">
        <ServerUrl>{url}</ServerUrl>
        <SRS>EPSG:3857</SRS>
    </Service>
    <DataWindow>
        <UpperLeftX>-20037508.34</UpperLeftX>
        <UpperLeftY>20037508.34</UpperLeftY>
        <LowerRightX>20037508.34</LowerRightX>
        <LowerRightY>-20037508.34</LowerRightY>
        <TileLevel>18</TileLevel>
        <TileCountX>1</TileCountX>
        <TileCountY>1</TileCountY>
        <YOrigin>top</YOrigin>
    </DataWindow>
    <Projection>EPSG:3857</Projection>
    <BlockSizeX>256</BlockSizeX>
    <BlockSizeY>256</BlockSizeY>
    <BandsCount>3</BandsCount>
    <Cache>
        <Path>../.gdalwmscache</Path>
        <Extension>.png</Extension>
    </Cache>
</GDAL_WMS>
'''

def export_image(image, bounds, name):
    coords_web = bounds.transform('EPSG:3857', 1).coordinates().getInfo()

    tile_fetcher = image.getMapId()['tile_fetcher']
    url = tile_fetcher.url_format.format(x='${x}', y='${y}', z='${z}')
    
    lr = coords_web[0][1][0], coords_web[0][1][1]
    ul = coords_web[0][3][0], coords_web[0][3][1]

    width = lr[0] - ul[0]
    height = ul[1] - lr[1]
        
    with open(name + '.xml', 'w') as f:
        xml = template.format(url=url)
        f.write(xml)
        
    !gdal_translate {name}.xml -projwin {ul[0]} {ul[1]} {lr[0]} {lr[1]} -outsize 2048 2048 {name}.png

In [3]:
# create a non-geodetic area of interest
bounds = ee.Geometry.Polygon([[
          [-122.75548673853245, 38.0411041965202],
          [-122.75548673853245, 37.64853352930806],
          [-122.11690641626683, 37.64853352930806],
          [-122.11690641626683, 38.0411041965202]]], None, False)

In [19]:
# export DEM
dem = ee.Image("USGS/NED").visualize(**{"min": 0, "max": 700})
export_image(image=dem, name='dem', bounds=bounds)

Input file size is 67108864, 67108864
0...10...20...30...40...50...60...70...80...90...100 - done.


In [5]:
# export water
water_polygons = ee.FeatureCollection("users/gena/OpenStreetMap/water")
water_lines = ee.FeatureCollection("users/gena/OpenStreetMap/waterways")
land = ee.Image("users/gena/land_polygons_image")

water_lines = water_lines.filterBounds(bounds)
water_polygons = water_polygons.filterBounds(bounds)
water = water_lines.merge(water_polygons)
water_image = ee.Image(0).paint(water, 1, 1)

water_ocean = land.unmask(1, False)
water_image = water_image.max(water_ocean)

water_image = water_image.visualize(**{'min': 0, 'max': 1, 'palette': ['000000', 'ffffff']}).clip(bounds)

# export to png
export_image(image=water_image, name='water', bounds=bounds)


Input file size is 67108864, 67108864
0...10...20...30...40...50...60...70...80...90...100 - done.
